In [138]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [139]:
og = pd.read_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/std_og.csv', index_col=0)
v3 = pd.read_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/std_B721.csv', index_col=0)
bigmhc = pd.read_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/std_bigmhc.csv', index_col=0)


In [140]:
og['src'] = 'HLAthenaV1'

In [141]:
def dedup(df):
    """remove contradictory pos/neg instances and then deduplicate"""
    # first copy all sorted duplicates and remove them from df
    dupidx = df.index.duplicated(keep=False)
    dups = df[dupidx].copy().sort_index()
    df = df[~dupidx]
    # then remove any contradictory pos/neg instances in the dups
    dups["rm"] = 0
    x = 0
    while x < len(dups)-1:
        rm = 0
        for y in range(x+1,len(dups)):
            if dups.index[x] != dups.index[y]:
                break
            if dups.tgt[x] != dups.tgt[y]:
                rm = 1
        dups.iloc[x:y,-1] = rm
        x = y
    dups = dups[dups.rm==0]
    # lastly deduplicate and concat
    dups = dups[~dups.index.duplicated()]
    dups.drop(["rm"], axis=1, inplace=True)
    return pd.concat((df,dups)).sort_index()

In [142]:
df = pd.concat([og,bigmhc,v3])

In [143]:
df = df.drop_duplicates(subset=['mhc','pep'], keep='first')

In [144]:
mhc_counts = pd.DataFrame(df['mhc'].value_counts())
rm_alleles = list(mhc_counts.loc[mhc_counts['mhc']<20].index)

In [145]:
df = df.loc[~df.mhc.isin(rm_alleles)]

In [146]:
df = df[~df.pep.str.contains('X')]

In [93]:
# a0202val = a0202.sample(600, random_state=1, replace=False)
# a0202train = a0202[~a0202.index.isin(a0202val.index)]
# a0202full[['tgt','len']].value_counts()
# # a0202 = df.loc[df['mhc']=='HLA-A*02:02']
# # any(['X' in seq for seq in a0202['pep']])
# # df
# # a0202['len'] = a0202.pep.apply(len)
# lencounts = {l:r.len for l,r in pd.DataFrame(a0202val.len.value_counts()).iterrows()}
# lencounts = {l:r.len for l,r in pd.DataFrame(a0202train.len.value_counts()).iterrows()}
# 
# decoys = pd.read_csv('/Users/cleoforman/PycharmProjects/hlathenav2/hlathena/data/model_training/decoys.txt')
# decoys['len'] = decoys.seq.apply(len)
# decoysdfs = []
# for l in lencounts:
#     decoysdfs.append(decoys.loc[decoys['len']==l].sample(lencounts[l]*1000, random_state=10, replace=True).reset_index(drop=True))
# decoydf = pd.concat(decoysdfs)
# # a0202
# decoydf['mhc'] = 'HLA-A*02:02'
# decoydf['tgt'] = 0
# decoydf['pep'] = decoydf['seq']
# def uid(df):
#     """set index as mhc_pep"""
#     df["uid"] = df.mhc + '_' + df.pep
#     return df.set_index("uid", drop=True)
# 
# decoydf = uid(decoydf)
# decoydf.len.value_counts()
# # a0202train.len.value_counts()
# 
# decoydf = decoydf[['mhc','pep','tgt']]
# decoydf['src'] = 'NA'
# decoydf['len'] = decoydf.pep.apply(len)
# a0202full = pd.concat([a0202train,decoydf])
# a0202full.to_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/A0202_x1000d_train.txt',index=None, sep=' ')

In [62]:
# df.drop('fold',axis=1,inplace=True)

/Users/cleoforman/PycharmProjects/hlathenav2/venv/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [119]:
df['len'] = df.pep.apply(len)


In [120]:
np.random.seed(0)
dfs = []
for g, d in df.groupby(['mhc','len']):
    # print(d)
    d['fold'] = np.random.choice(6, len(d), replace=True)
    print(g)
    print(d['fold'].value_counts())
    dfs.append(d)
    # break

('HLA-A*01:01', 8)
3    78
0    72
5    63
1    55
4    54
2    53
Name: fold, dtype: int64
('HLA-A*01:01', 9)
3    426
5    404
1    392
2    391
4    389
0    370
Name: fold, dtype: int64
('HLA-A*01:01', 10)
0    309
5    309
1    306
4    299
3    299
2    292
Name: fold, dtype: int64
('HLA-A*01:01', 11)
0    220
2    200
5    198
3    197
1    192
4    192
Name: fold, dtype: int64
('HLA-A*02:01', 8)
1    108
4     92
0     84
3     82
5     80
2     65
Name: fold, dtype: int64
('HLA-A*02:01', 9)
1    1373
2    1327
0    1325
5    1316
3    1314
4    1313
Name: fold, dtype: int64
('HLA-A*02:01', 10)
3    412
5    401
4    401
2    400
0    399
1    370
Name: fold, dtype: int64
('HLA-A*02:01', 11)
2    330
3    326
5    312
0    302
4    299
1    296
Name: fold, dtype: int64
('HLA-A*02:02', 8)
3    23
2    16
0    14
1    12
4    11
5     8
Name: fold, dtype: int64
('HLA-A*02:02', 9)
4    540
2    493
3    489
5    488
1    478
0    463
Name: fold, dtype: int64
('HLA-A*02:02', 10)
5 

In [125]:
valdf = fold_df.loc[fold_df.fold==5][["mhc","pep","tgt","src","len"]]

In [128]:
valdf.to_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/new_data/MS_v2_panlen_seed_1_hits_valid.txt', sep='' '')

,mhc,pep,tgt,src,len
uid,,,,,
HLA-A*01:01_ADKVHLMV,HLA-A*01:01,ADKVHLMV,1,BigMHC,8
HLA-A*01:01_AQNEFSLY,HLA-A*01:01,AQNEFSLY,1,BigMHC,8
HLA-A*01:01_ATEPVIAF,HLA-A*01:01,ATEPVIAF,1,BigMHC,8
HLA-A*01:01_ATMMKMSL,HLA-A*01:01,ATMMKMSL,1,BigMHC,8
HLA-A*01:01_DGKSRWFY,HLA-A*01:01,DGKSRWFY,1,BigMHC,8
...,...,...,...,...,...
HLA-G*01:04_KVPPPWLIAM,HLA-G*01:04,KVPPPWLIAM,1,HLAthenaV1,10
HLA-G*01:04_NSSRLEPLVL,HLA-G*01:04,NSSRLEPLVL,1,HLAthenaV1,10
HLA-G*01:04_PLSMAPPLSL,HLA-G*01:04,PLSMAPPLSL,1,HLAthenaV1,10


In [149]:
cv1_train = fold_df.loc[fold_df['fold'].isin([0,1,2])]
cv1_test = fold_df.loc[fold_df['fold']==3]
cv1_valid = fold_df.loc[fold_df['fold']==4]

In [151]:
print(len(cv1_train)/len(fold_df))
print(len(cv1_test)/len(fold_df))
print(len(cv1_valid)/len(fold_df))


0.5985285118615605
0.20048670110575076
0.20098478703268866


In [148]:
ms8 = pd.read_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/ogdata/MS8_dat_dummy_seed_757_5cv_x10p_train_test_valid.txt', sep=' ')
ms9 = pd.read_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/ogdata/MS9_dat_dummy_seed_757_5cv_x10p_train_test_valid.txt', sep=' ')
ms10 = pd.read_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/ogdata/MS10_dat_dummy_seed_757_5cv_x10p_train_test_valid.txt', sep=' ')
ms11 = pd.read_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/ogdata/MS11_dat_dummy_seed_757_5cv_x10p_train_test_valid.txt', sep=' ')
og = pd.concat([ms8,ms9,ms10,ms11])

In [150]:
cv1 = og.loc[og.fold==0]

In [157]:
pep_df = og
fold=0
train_df = pep_df.loc[(pep_df["set"] == "train") & (pep_df["fold"] == fold)].reset_index(drop=True)
test_df = pep_df.loc[(pep_df["set"] == "test") & (pep_df["fold"] == fold)].reset_index(drop=True)
valid_df = pep_df.loc[(pep_df["set"] == "valid") & (pep_df["fold"] == fold)].reset_index(drop=True)


In [161]:
pep_df.to_csv('/Users/cleoforman/PycharmProjects/hlathenav2/models/data/ogdata/MS_panlen_seed_757_5cv_x10p_train_test_valid.txt', sep=' ', index=None)

In [137]:
hits.loc[hits.set=='valid'].fold.value_counts()

0    275820
4    274440
2    272730
1    272080
3    271190
Name: fold, dtype: int64

In [136]:
hits_dedup.loc[hits_dedup.set=='valid'].fold.value_counts()

2    24632
1    22630
0    19152
4    18767
3    15816
Name: fold, dtype: int64

In [134]:
hits_dedup = hits.drop_duplicates(subset=['allele','seq','set'])

In [147]:
print(len(hits_dedup.loc[hits_dedup['set']=='valid'])/len(hits_dedup))
print(len(hits_dedup.loc[hits_dedup['set']=='train'])/len(hits_dedup))
print(len(hits_dedup.loc[hits_dedup['set']=='test'])/len(hits_dedup))


0.1596957506982627
0.6398135623986275
0.20049068690310984


In [ ]:
# for each fold:
    # for each allele,len:
        # 1/5 test
        # 3/5 train
        # 1/5 validate
# split into 5ths and assign?